# Assignment #8 - Data Gathering and Warehousing - DSSA-5102

Instructor: Melissa Laurino</br>
Spring 2025</br>

Name: Thinh Le
</br>
Date: April 05, 2025
<br>
<br>
**At this time in the semester:** <br>
- We have explored a dataset. <br>
- We have cleaned our dataset. <br>
- We created a Github account with a repository for this class and included a metadata read me file about our data. <br>
- We introduced general SQL syntax, queries, and applications in Python.<br>
- Created our own databases from scratch using MySQL Workbench and Python with SQLAlchemy on our local server and locally on our machine.
- Populated our databases with the data we cleaned at the start of the semester.
<br>

Now we will **JOIN** our knowledge and tables to answer more complex questions about our dataset! We will practice joining tables and understand the importance of using different commands.<br>

JOIN statements are used to combine results from two or more tables based on a related column between them.<br>

Review the powerpoint and readings specified on Blackboard.<br>

In the event your database does not meet the requirements below to answer the question, please use the database provided in Assignment #4 and #5. Remember to credit your data source. <br>

Follow the instructions below to complete the assignment. Write your question you are answering with your data query and visualize your results in a way that fits your data. <br>
Be sure to comment **all** code and answer **all** questions in markdown for full credit.<br>

**Data origin:** SQL murder mystery

### Import libraries

In [9]:
from sqlalchemy import create_engine, inspect, text # Object Relational Mapper
import pandas as pd # Python data manilpulation

### Create a new engine

In [10]:
# How to connect to our .db file using library SQLAlchemy
# Notice that we are not specifying a host, password, or server since this .db file is stored locally.
db_path = "sql-murder-mystery.db"
# Use the create_engine function to connect to the database.
engine = create_engine(f"sqlite:///{db_path}?charset=utf8mb4")
engine

Engine(sqlite:///sql-murder-mystery.db?charset=utf8mb4)

#### INNER JOIN (or JOIN)
Display matching records from TWO tables! Choose to combine two tables using inner join. <br>
Write your question you are answering with your data query and visualize your results. <br>
<br>
**Example Question:** How many persons have driver license?

**What tables are we joining?** person and drivers_license

In [11]:
with engine.connect() as connection:
    # Create query
    query = text("""
        SELECT COUNT(*)
        FROM person
        INNER JOIN drivers_license
        ON person.license_id = drivers_license.id
    """)
    # Read query result
    persons_have_driver_license_count = pd.read_sql(query, connection)

# Print the results
persons_have_driver_license_count

,COUNT(*)
0,10006


Count how many persons in person table:

In [12]:
with engine.connect() as connection:
    # Create query
    query = text("""
        SELECT COUNT(*)
        FROM person
    """)
    # Read query result
    persons_count = pd.read_sql(query, connection)

# Print the results
persons_count

,COUNT(*)
0,10011


So there are 10006/10011 persons have driver license.

#### LEFT JOIN
<br>
Returns ALL records from the left table and matching records from the right table. Write your question you are answering with your data query and visualize your results.. 
<br><br>
**Question:** Is using left join on person table with drivers_license table return the same result with count all rows from person table?
<br>
**What tables are we joining?** person and drivers_license

In [13]:
with engine.connect() as connection:
    # Create query
    query = text("""
        SELECT COUNT(*)
        FROM person
        LEFT JOIN drivers_license
        ON person.license_id = drivers_license.id
    """)
    # Read query result
    persons_left_join_driver_license_count = pd.read_sql(query, connection)

# Print the results
persons_left_join_driver_license_count

,COUNT(*)
0,10011


The answer is yes.

#### RIGHT JOIN
<br>
Returns ALL records from the right table and matching records from the left table. Write your question you are answering with your data query and visualize your results.. <br>
<br>
**Question:** How many people got interviewed?
<br>
**What tables are we joining?** person and interview

In [14]:
with engine.connect() as connection:
    # Create query
    query = text("""
        SELECT COUNT(*)
        FROM person
        RIGHT JOIN interview
        ON person.id = interview.person_id
    """)
    # Read query result
    interviews_right_join = pd.read_sql(query, connection)

# Print the results
interviews_right_join

,COUNT(*)
0,4991


#### FULL JOIN or UNION of RIGHT JOIN and LEFT JOIN
<br>
Can answer multiple objectives at the same time! Not recommended for large databases. Results may slow your machine or quit before finishing. Write your question you are answering with your data query and visualize your results. <br>
<br>
**Question:** Get names of people who interviewed and not. If they got interviewed, get their transcript.
<br>
**What tables are we joining?** person and interview

In [21]:
with engine.connect() as connection:
    # Create query
    query = text("""
        SELECT person.id, person.name, interview.transcript
        FROM person
        FULL JOIN interview
        ON person.id = interview.person_id
    """)
    # Read query result
    persons_and_interviews = pd.read_sql(query, connection)

# Print the results
persons_and_interviews

,id,name,transcript
0,10000,Christoper Peteuil,None
1,10007,Kourtney Calderwood,CHAPTER IV. The Rabbit Sends in a Little Bill\n
2,10010,Muoi Cary,None
3,10016,Era Moselle,\n
4,10025,Trena Hornby,\n
...,...,...,...
10006,99936,Luba Benser,"‘Oh! So Bill’s got to come down the chimney, h..."
10007,99941,Roxana Mckimley,None
10008,99965,Cherie Zeimantz,"said ‘No, never’) ‘--so you can have no idea w..."
10009,99982,Allen Cruse,None


If the person_id doesn't exist in the interview table, the transcript is None. To verify the result, we can try to search for person_id = 10000, which has None in transcript column:

In [24]:
with engine.connect() as connection:
    # Create query
    query = text("""
        SELECT *
        FROM interview
        WHERE interview.person_id = '10000'

    """)
    # Read query result
    verify_result = pd.read_sql(query, connection)

# Print the results
verify_result

,person_id,transcript


As expected, there is no result returned. This means the full join worked as expected.

In [28]:
# Close your connection :)
connection.close()
engine.dispose()